# Two layer net

## 背景

一般来说网络的层数越多其表达能力越强，因为其参数更多，能表达出的状态信息更多，所以表达能力越强。多层神经网络可以应对更加复杂的问题，这一节我们先从一个小例子开始：构建一个两层神经网络

## 准备工作

参考[SoftmaxLinearClassifier](https://thoughtworksinc.github.io/DeepLearning.scala/demo/SoftmaxLinearClassifier.html#准备工作)

## 构建神经网络

import所需的依赖

In [12]:
import $plugin.$ivy.`com.thoughtworks.implicit-dependent-type::implicit-dependent-type:2.0.0`

import $ivy.`com.thoughtworks.deeplearning::differentiableany:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiablenothing:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiableseq:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiabledouble:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiablefloat:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiablehlist:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiablecoproduct:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiableindarray:1.0.0-RC7`
import $ivy.`org.rauschig:jarchivelib:0.5.0`

import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.0`

import java.io.{FileInputStream, InputStream}


import com.thoughtworks.deeplearning
import org.nd4j.linalg.api.ndarray.INDArray
import com.thoughtworks.deeplearning.DifferentiableHList._
import com.thoughtworks.deeplearning.DifferentiableDouble._
import com.thoughtworks.deeplearning.DifferentiableINDArray._
import com.thoughtworks.deeplearning.DifferentiableAny._
import com.thoughtworks.deeplearning.DifferentiableINDArray.Optimizers._
import com.thoughtworks.deeplearning.DifferentiableINDArray.Layers.Weight
import com.thoughtworks.deeplearning.{
  DifferentiableHList,
  DifferentiableINDArray,
  Layer,
  Symbolic
}
import com.thoughtworks.deeplearning.Layer.Batch
import com.thoughtworks.deeplearning.Symbolic.Layers.Identity
import com.thoughtworks.deeplearning.Symbolic._
import com.thoughtworks.deeplearning.Poly.MathFunctions._
import com.thoughtworks.deeplearning.Poly.MathMethods./
import com.thoughtworks.deeplearning.Poly.MathOps
import org.nd4j.linalg.api.ndarray.INDArray
import org.nd4j.linalg.cpu.nativecpu.NDArray
import org.nd4j.linalg.factory.Nd4j
import org.nd4j.linalg.indexing.{INDArrayIndex, NDArrayIndex}
import org.nd4j.linalg.ops.transforms.Transforms
import org.nd4s.Implicits._
import shapeless._

import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._

import scala.collection.immutable.IndexedSeq

pprintConfig() = pprintConfig().copy(height = 5)//减少输出的行数，避免页面输出太长

import $file.ReadCIFAR10ToNDArray,ReadCIFAR10ToNDArray._
import $file.Utils,Utils._

Compiling ReadCIFAR10ToNDArray.sc
Compiling Utils.sc


import $plugin.$                                                                             


import $ivy.$                                                           

import $ivy.$                                                               

import $ivy.$                                                           

import $ivy.$                                                              

import $ivy.$                                                             

import $ivy.$                                                             

import $ivy.$                                                                 

import $ivy.$                                                                

import $ivy.$                               


import $ivy.$                                             


import java.io.{FileInputStream, InputStream}



import com.thoughtworks.deeplearning

import org.nd4j.linalg.api.ndarray.INDArray

import com.thoughtworks.deeplearning.Differentiabl

从CIFAR10 database中读取测试数据的图片和标签信息

In [13]:
//CIFAR10中的图片共有10个分类(airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck)
val NumberOfClasses: Int = 10

//加载测试数据，我们读取100条作为测试数据
val testNDArray =
  ReadCIFAR10ToNDArray.readFromResource("/cifar-10-batches-bin/test_batch.bin", 100)

NumberOfClasses: Int = 10
testNDArray: INDArray :: INDArray :: HNil = [[0.62, 0.62, 0.64, 0.65, 0.62, 0.61, 0.63, 0.62, 0.62, 0.62, 0.63, 0.62, 0.63, 0.65, 0.66, 0.66, 0.65, 0.63, 0.62, 0.62, 0.61, 0.58, 0.59, 0.58, 0.58, 0.56, 0.55, 0.55, 0.56, 0.54, 0.49, 0.45, 0.59, 0.59, 0.62, 0.65, 0.63, 0.62, 0.64, 0.63, 0.64, 0.61, 0.61, 0.62, 0.64, 0.66, 0.67, 0.67, 0.66, 0.62, 0.60, 0.59, 0.57, 0.54, 0.55, 0.55, 0.58, 0.57, 0.57, 0.55, 0.56, 0.53, 0.49, 0.46, 0.59, 0.59, 0....

分离和处理图像和标签数据

In [14]:
val testData = testNDArray.head

val testExpectResult = testNDArray.tail.head

val vectorizedTestExpectResult = Utils.makeVectorized(testExpectResult, NumberOfClasses)

testData: INDArray = [[0.62, 0.62, 0.64, 0.65, 0.62, 0.61, 0.63, 0.62, 0.62, 0.62, 0.63, 0.62, 0.63, 0.65, 0.66, 0.66, 0.65, 0.63, 0.62, 0.62, 0.61, 0.58, 0.59, 0.58, 0.58, 0.56, 0.55, 0.55, 0.56, 0.54, 0.49, 0.45, 0.59, 0.59, 0.62, 0.65, 0.63, 0.62, 0.64, 0.63, 0.64, 0.61, 0.61, 0.62, 0.64, 0.66, 0.67, 0.67, 0.66, 0.62, 0.60, 0.59, 0.57, 0.54, 0.55, 0.55, 0.58, 0.57, 0.57, 0.55, 0.56, 0.53, 0.49, 0.46, 0.59, 0.59, 0....
testExpectResult: INDArray = [3.00, 8.00, 8.00, 0.00, 6.00, 6.00, 1.00, 6.00, 3.00, 1.00, 0.00, 9.00, 5.00, 7.00, 9.00, 8.00, 5.00, 7.00, 8.00, 6.00, 7.00, 0.00, 4.00, 9.00, 5.00, 2.00, 4.00, 0.00, 9.00, 6.00, 6.00, 5.00, 4.00, 5.00, 9.00, 2.00, 4.00, 1.00, 9.00, 5.00, 4.00, 6.00, 5.00, 6.00, 0.00, 9.00, 3.00, 9.00, 7.00, 6.00, 9.00, 8.00, 0.00, 3.00, 8.00, 8.00, 7.00, 7.00, 4.00, 6.00, 7.00, 3.00, 6.00, 3.00, 6.00, 2.00, 1.0...
vectorizedTestExpectResult: INDArray = [[0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
 [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.

设置学习率

In [15]:
implicit val optimizerFactory = new DifferentiableINDArray.OptimizerFactory {
  override def ndArrayOptimizer(weight: Weight): Optimizer = {
    new LearningRate with L2Regularization {

      var learningRate = 0.001

      override protected def currentLearningRate(): Double = {
        learningRate *= 0.9995
        learningRate
      }

      override protected def l2Regularization: Double = 0.03
    }
  }
}

optimizerFactory: AnyRef with OptimizerFactory = $sess.cmd14Wrapper$Helper$$anon$2@2ed09995

编写第一层神经网络

In [16]:
def fullyConnectedThenRelu(inputSize: Int, outputSize: Int)(
    implicit row: INDArray @Symbolic): INDArray @Symbolic = {
  val w = (Nd4j.randn(inputSize, outputSize) / math.sqrt(outputSize / 2.0)).toWeight * 0.1
  val b = Nd4j.zeros(outputSize).toWeight
  max((row dot w) + b, 0.0)
}

defined function fullyConnectedThenRelu

编写softmax函数

In [17]:
def softmax(implicit scores: INDArray @Symbolic): INDArray @Symbolic = {
  val expScores = exp(scores)
  expScores / expScores.sum(1)
}

defined function softmax

编写第二层神经网络

In [18]:
def fullyConnectedThenSoftmax(inputSize: Int, outputSize: Int)(
    implicit row: INDArray @Symbolic): INDArray @Symbolic = {
  val w = (Nd4j.randn(inputSize, outputSize) / math.sqrt(outputSize)).toWeight //* 0.1
  val b = Nd4j.zeros(outputSize).toWeight
  softmax.compose((row dot w) + b)
}

defined function fullyConnectedThenSoftmax

将上面的两层神经网络组合起来

In [19]:
def hiddenLayer(implicit input: INDArray @Symbolic): INDArray @Symbolic = {
  val layer0 = fullyConnectedThenRelu(3072, 500).compose(input)
  fullyConnectedThenSoftmax(500, 10).compose(layer0)
}

val predictor = hiddenLayer

defined function hiddenLayer
predictor: (Symbolic.To[INDArray]{type OutputData = org.nd4j.linalg.api.ndarray.INDArray;type OutputDelta = org.nd4j.linalg.api.ndarray.INDArray;type InputData = org.nd4j.linalg.api.ndarray.INDArray;type InputDelta = org.nd4j.linalg.api.ndarray.INDArray})#@ = Compose(Compose(MultiplyINDArray(Exp(Identity()),Reciprocal(Sum(Exp(Identity()),WrappedArray(1)))),PlusINDArray(Dot(Identity(),Weight([[0.05, 0.27, 0.34, 0.42, -0.36, -0.13, 0.09, 0.86, 0.01, -0.43],
 [0.72, 0.38, 0.30, -0.05, -0.07, -0.46, 0.01, -0.18, 0.24, -0.32],
 [-0.08, -0.58, 0.10, 0.34, 0.18, -0.47, -0.09, -0.07, -0.14, -0.27],
...

编写损失函数Loss Function

In [20]:
def crossEntropy(
    implicit pair: (INDArray :: INDArray :: HNil) @Symbolic): Double @Symbolic = {
  val score = pair.head
  val label = pair.tail.head
  -(label * log(score * 0.9 + 0.1) + (1.0 - label) * log(1.0 - score * 0.9)).mean
}

defined function crossEntropy

组合输入层和隐含层

In [21]:
def network(
   implicit pair: (INDArray :: INDArray :: HNil) @Symbolic): Double @Symbolic = {
  val input = pair.head
  val label = pair.tail.head
  val score: INDArray @Symbolic = predictor.compose(input)
  val hnilLayer: HNil @Symbolic = HNil
  crossEntropy.compose(score :: label :: hnilLayer)
}

val trainer = network

defined function network
trainer: (Symbolic.To[Double]{type OutputData = Double;type OutputDelta = Double;type InputData = shapeless.::[org.nd4j.linalg.api.ndarray.INDArray,shapeless.::[org.nd4j.linalg.api.ndarray.INDArray,shapeless.HNil]];type InputDelta = shapeless.:+:[org.nd4j.linalg.api.ndarray.INDArray,shapeless.:+:[org.nd4j.linalg.api.ndarray.INDArray,shapeless.CNil]]})#@ = Compose(Negative(ReduceMean(PlusINDArray(MultiplyINDArray(Head(Tail(Identity())),Log(PlusDouble(MultiplyDouble(Head(Identity()),Literal(0.9)),Literal(0.1)))),MultiplyINDArray(PlusDouble(Negative(Head(Tail(Identity()))),Literal(1.0)),Log(PlusDouble(Negative(MultiplyDouble(Head(Identity()),Literal(0.9))),Literal(1.0))))))),HCons(Compose(Compose(Compose(MultiplyINDArray(Exp(Identity()),Reciprocal(Sum...

训练神经网络并观察每次训练loss的变化，loss的变化趋势是降低，但是不是每次都降低(前途是光明的，道路是曲折的)

In [22]:
val lossSeq = for (_ <- 0 until 2000) yield {
  val trainNDArray = ReadCIFAR10ToNDArray.getSGDTrainNDArray(256)
  val loss = network.train(
    trainNDArray.head :: Utils.makeVectorized(trainNDArray.tail.head, NumberOfClasses) :: HNil)
  println(loss)
  loss
}
val plot = Seq(
  Scatter(lossSeq.indices, lossSeq)
)

plot.plot(
  title = "loss by time"
)

0.26728835105896
0.26156439781188967
0.26684961318969724
0.27567334175109864
0.27993159294128417
0.29596452713012694
0.33901519775390626
0.41857023239135743
0.40464391708374026
0.380973744392395
0.41316871643066405
0.3990656614303589
0.4092988967895508
0.4171145439147949
0.41519870758056643
0.41823863983154297
0.42770891189575194
0.4150687217712402
0.4128608703613281
0.4075505256652832
0.4220388412475586
0.419892692565918
0.41027207374572755
0.41840476989746095
0.399689769744873
0.39670519828796386
0.4107173442840576
0.41579599380493165
0.4021256923675537
0.41770057678222655
0.39409627914428713
0.39914898872375487
0.35069024562835693
0.424986743927002
0.4324056148529053
0.4094971179962158
0.4087658882141113
0.34469265937805177
0.4099062442779541
0.40074429512023924
0.3953963279724121
0.42728090286254883
0.4062638282775879
0.4172062873840332
0.41359500885009765
0.41883258819580077
0.4134968757629395
0.43782811164855956
0.4187058448791504
0.4042646408081055
0.38996362686157227
0.42962098

lossSeq: IndexedSeq[Symbolic.To.<refinement>.this.type.OutputData] = Vector(
  0.26728835105896,
  0.26156439781188967,
  0.26684961318969724,
  0.27567334175109864,
...
plot: Seq[Scatter] = List(
  Scatter(
    Some(
      Doubles(
        Vector(
...
res21_2: String = "plot-44406801"

13.使用训练后的神经网络判断测试数据的标签

In [23]:
val result = predictor.predict(testData)
println(s"result: $result") //输出判断结果

result: [[0.01, 0.12, 0.15, 0.35, 0.02, 0.12, 0.18, 0.00, 0.05, 0.00],
 [0.05, 0.11, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.45, 0.39],
 [0.22, 0.07, 0.00, 0.01, 0.00, 0.00, 0.00, 0.00, 0.63, 0.06],
 [0.43, 0.03, 0.02, 0.01, 0.01, 0.01, 0.00, 0.01, 0.48, 0.01],
 [0.00, 0.01, 0.11, 0.06, 0.51, 0.06, 0.17, 0.07, 0.01, 0.00],
 [0.00, 0.05, 0.02, 0.24, 0.02, 0.17, 0.47, 0.01, 0.00, 0.01],
 [0.00, 0.04, 0.01, 0.66, 0.00, 0.23, 0.06, 0.00, 0.00, 0.00],
 [0.00, 0.00, 0.24, 0.03, 0.40, 0.03, 0.23, 0.06, 0.00, 0.00],
 [0.02, 0.03, 0.29, 0.14, 0.14, 0.31, 0.02, 0.05, 0.01, 0.00],
 [0.02, 0.61, 0.00, 0.01, 0.00, 0.00, 0.00, 0.00, 0.29, 0.08],
 [0.22, 0.01, 0.00, 0.01, 0.01, 0.01, 0.00, 0.00, 0.73, 0.01],
 [0.00, 0.25, 0.00, 0.00, 0.00, 0.00, 0.00, 0.01, 0.04, 0.70],
 [0.00, 0.26, 0.04, 0.16, 0.07, 0.15, 0.27, 0.02, 0.02, 0.01],
 [0.01, 0.39, 0.01, 0.14, 0.00, 0.04, 0.33, 0.01, 0.04, 0.03],
 [0.05, 0.22, 0.07, 0.03, 0.03, 0.01, 0.02, 0.15, 0.14, 0.29],
 [0.07, 0.00, 0.19, 0.02, 0.13, 0.03, 0.02, 0.0

result: Symbolic.To.<refinement>.this.type.OutputData = [[0.01, 0.12, 0.15, 0.35, 0.02, 0.12, 0.18, 0.00, 0.05, 0.00],
 [0.05, 0.11, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.45, 0.39],
 [0.22, 0.07, 0.00, 0.01, 0.00, 0.00, 0.00, 0.00, 0.63, 0.06],
 [0.43, 0.03, 0.02, 0.01, 0.01, 0.01, 0.00, 0.01, 0.48, 0.01],
 [0.00, 0.01, 0.11, 0.06, 0.51, 0.06, 0.17, 0.07, 0.01, 0.00],
...

15.判断神经网络对测试数据分类判断的正确率，正确率应该在50%左右。

In [24]:
val right = Utils.getAccuracy(result, testExpectResult)
println(s"the result is $right %")

the result is 49.0 %


right: Double = 49.0


16.[完整代码](https://github.com/izhangzhihao/deeplearning-tutorial/blob/master/src/main/scala/com/thoughtworks/deeplearning/tutorial/TwoLayerNet.scala)